<a href="https://colab.research.google.com/github/Yolantele/ML-data-clasifier/blob/master/SpaCy_ML_Classifier_for_Waste_Data_Augmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

####SpaCy_ML_Classifier_for_Waste_Data_Augmentation

In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [1]:
# mount data from drive
from google.colab import drive
drive.mount('/content/drive')
path = '/content/drive/My Drive/data/'


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
!pip install -U spacy==2.2.2
!pip install pandas

In [3]:
import string
import spacy
from spacy.lang.en.stop_words import STOP_WORDS
import pandas as pd
from spacy.lang.en import English
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.base import TransformerMixin


# References:
# https://www.dataquest.io/blog/tutorial-text-classification-in-python-using-spacy/

spacy.prefer_gpu()
nlp = spacy.load("en_core_web_sm")

In [134]:
materials = pd.read_csv(path + '/enMaterialData.csv')
# or use test data frame where material field is empty
materials_test = pd.read_csv(path + '/enWithoutMaterialData.csv')

df = materials
df.head()
# df.info()
# df.description 
# df.description + df.euralDescription


,reason,origin,color,state,size,consistency,otherCode,material4,material3,material2,material,mType,composite2,composite1,cType,indirectProduct,directProduct,pType,mixedOrPure,cleanOrDirty,euralDescription,euralCode,description,/0
0,NaN,NaN,NaN,NaN,NaN,slurry,NaN,NaN,NaN,oil,organic material,Soy,NaN,dry,NaN,dry,NaN,NaN,1.0,NaN,material unsuitable for consumption or processing,20304,Soyadroes technically,NaN
1,NaN,NaN,NaN,NaN,NaN,vast,NaN,NaN,NaN,NaN,wood,branches,NaN,NaN,NaN,wood,NaN,NaN,0.0,NaN,waste from forestry,20107,Branches,NaN
2,NaN,NaN,NaN,NaN,NaN,vast,NaN,NaN,NaN,NaN,cocoa,caps,NaN,NaN,NaN,cocoa,NaN,NaN,0.0,NaN,material unsuitable for consumption or processing,20304,Cocoa shells,NaN
3,NaN,NaN,NaN,debris,NaN,vast,NaN,NaN,NaN,NaN,sand,NaN,NaN,debris,NaN,debris,NaN,NaN,1.0,1.0,"mixtures of concrete, stones, tiles or ceramic...",170107,Debris with Sand,NaN
4,NaN,NaN,NaN,NaN,NaN,vast,NaN,NaN,NaN,NaN,organic material,Soy,NaN,NaN,NaN,organic material,NaN,NaN,0.0,NaN,material unsuitable for consumption or processing,20304,soya,NaN


###Tokening the Data With spaCy

Now that we know what we’re working with, let’s create a custom tokenizer function using spaCy. We’ll use this function to automatically strip information we don’t need, like stopwords and punctuation, from each review.

In [225]:
from spacy.lang.en.stop_words import STOP_WORDS

parser = English()
punctuations = '''!()-[]{};:'"\,<>./?@#$%^&*_~'''
stopwords = STOP_WORDS

def spacy_tokenize(sentence):
    # Creating our token object, which is used to create documents with linguistic annotations.
    sentence = sentence.strip().lower()
    tokens = parser(sentence)

    # Lemmatizing each token and converting each token into lowercase
    tokens = [ word.lemma_.lower().strip() if word.lemma_ != "-PRON-" else word.lower_ for word in tokens ]

    # Removing stop words
    tokens = [ word for word in mytokens if word not in stopwords and word not in punctuations ]

    # return preprocessed list of tokens
    return tokens


###Get all Nouns from description


#### unique materials list

In [315]:
# 1. unique existing meterials list
MATERIALS = [
  'organic material',
  'organic',
  'polypropylene',
  'wood',
  'cocoa',
  'sand',
  'starch',
  'water',
  'grind',
  'ceramics',
  'collect',
  'metal',
  'plastic',
  'plastics',
  'soy',
  'bentonite',
  'dough',
  'oil',
  'oils',
  'melamine',
  'ground',
  'know',
  'most',
  'pvc',
  'fatty acids',
  'pom',
  'asbestos',
  'cast',
  'stones',
  'hdpe',
  'paper',
  'snails',
  'glass',
  'ldpe',
  'bold',
  'pp',
  'rvs',
  'iron',
  'stories',
  'eps',
  'right now',
  'pet',
  'sintels',
  'minerals',
  'beer',
  'up',
  'carton',
  'section',
  'molasses',
  'tyleen',
  'avi-bodemas',
  'pc',
  'pmma',
  'carbon',
  'bitumen',
  'perlite',
  'teer',
  'resin',
  'glass wool',
  'aluminum',
  'rock wool',
  'wool',
  'rubber',
  'sugar',
  'bread',
  'pfos',
  'optical fiber',
  'insulation material',
  'insulation',
  'steel',
  'dill',
  'ps',
  'on',
  'compost',
  'wine',
  'turf',
  'cork',
  'foam rubber',
  'cement',
  'pa',
  'nylon',
  'messing',
  'believe',
  'zinc',
  'non-iron',
  'flint',
  'pcfic',
  'kwik',
  'syrup',
  'lime',
  'solvents',
  'zeolite',
  'porcelain',
  'polyester',
  'snail wool',
  'fly ash',
  'beryllium',
  'nickel',
  'glycerine',
  'sodium hypochlorite',
  'acid',
  'urea',
  'cheese',
  'dioxin',
  'cyaniden',
  'log',
  'vocl',
  'activated carbon',
  'wrap',
  'vinasse',
  'shells',
  'pcb',
  'magnesium',
  'nox',
  'fipronil',
  'koolas',
  'fiber',
  'dog',
  'paraffin',
  'styrene butadiene rubber',
  'styrene',
  'butadiene',
  'cadmium',
  'then',
  'glycol',
  'inorganic material',
  'school',
  'formaline',
  'salt',
  'salts',
  'ocb',
  'pak',
  'mo',
  'silicon',
  'stone',
  'chocolate',
  'chrome',
  'soil',
  'soya',
  'foil',
  'concrete',
  'breadcrumb',
  'breadcrumbs',
  'plaster',
  'aw',
  'concrete',
]

#### extract unique materials 

In [321]:
import spacy
import re
import string

parser = spacy.load('en_core_web_sm')

def extract_materials(row):
  sentence = str(df.iloc[row]['description'])
  sentence = sentence.strip().lower()
  sentence = re.sub(r'\d+', '', sentence) # removes numbers 
  sentence = re.sub(r'[^\w\s]','',sentence) # removes punctuations

  print('sentence----', sentence)
  doc = parser(sentence)

  
  for np in doc.noun_chunks:
    for token in np:
      if token.pos_ is 'NOUN' or token.pos_ is 'PROPN':
        if token.text in MATERIALS or token.lemma_ in MATERIALS:
          print('rich mat:', np)


  #only material noun:    
  index = [1] # iterate through sentence once
  for idxValue in index:
    for token in doc:
      # print(token, '--', token.pos_)
      if token.pos_ is 'NOUN' or token.pos_ is 'PROPN' or token.pos_ is 'ADJ' or token.pos_ is 'INTJ':
        if token.text in MATERIALS or token.lemma_ in MATERIALS:
          print('material:',  token.text)
          
  print()

extract_materials(919)
extract_materials(929)
extract_materials(939)
extract_materials(949)

sentence---- rvs albermarle
rich mat: rvs
material: rvs

sentence---- clean concrete debris
material: concrete

sentence---- bbk aw starting
material: aw

sentence---- land oude doelenkade  hoorn



In [319]:
# Visualize Entities
from spacy import displacy
import IPython

row = 33
description = str(df.iloc[row]['description'])
doc = parser(description)
print('entities----', [(ent.text, ent.label_, spacy.explain(ent.label_)) for ent in doc.ents])
print(spacy.explain('INTJ'))

visualize = displacy.render(doc, style="ent")

IPython.display.HTML(visualize)



entities---- [('melamine-urea / formalin co-polymer', 'ORG', 'Companies, agencies, institutions, etc.')]
interjection


###Vectorization Feature Engineering (TF-IDF) , Bag of Words and N-grams

Classifying text we end up with text snippets with their respective labels. But in machine learning model we need to convert into numeric representation

TF-IDF -Term Frequency-Inverse Document Frequency - simply a way of normalizing our Bag of Words(BoW) by looking at each word’s frequency in comparison to the document frequency.

N-grams - combinations of adjacent words in a given text. For example "who will win"
- when n = 1, becomes "who", "will", "win"
- when n = 2 , becomes "who will", "will win" etc. 

In [ ]:
#bag of words vector
bow_vector = CountVectorizer(tokenizer=spacy_tokenize, ngram_range=(1,1))
print(bow_vector)

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                lowercase=True, max_df=1.0, max_features=None, min_df=1,
                ngram_range=(1, 1), preprocessor=None, stop_words=None,
                strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=<function spacy_tokenize at 0x7f5ebe885e18>,
                vocabulary=None)


In [ ]:
tfidf_vector = TfidfVectorizer(tokenizer=spacy_tokenize)
print(tfidf_vector)

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.float64'>, encoding='utf-8',
                input='content', lowercase=True, max_df=1.0, max_features=None,
                min_df=1, ngram_range=(1, 1), norm='l2', preprocessor=None,
                smooth_idf=True, stop_words=None, strip_accents=None,
                sublinear_tf=False, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=<function spacy_tokenize at 0x7f5ebe885e18>,
                use_idf=True, vocabulary=None)


### Splitting The Data into Training and Validation Sets


In [ ]:
from sklearn.model_selection import train_test_split

X = df['description'] # the features we want to analyze
ylabels = df['material'] # the labels, or answers, we want to test against

X_train, X_test, y_train, y_test = train_test_split(X, ylabels, test_size=0.3)

print(X)

### Creating a Pipeline and Generating the Model

In [ ]:
# Creating Logistic Regression Classifier
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline

classifier = LogisticRegression()

# Create pipeline using Bag of Words
pipe = Pipeline([("cleaner", predictors()),
                 ('vectorizer', bow_vector),
                 ('classifier', classifier)])

# model generation
pipe.fit(X_train,y_train)

Pipeline(memory=None,
         steps=[('cleaner', <__main__.predictors object at 0x7f5eb771e080>),
                ('vectorizer',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 t...\\b\\w\\w+\\b',
                                 tokenizer=<function spacy_tokenize at 0x7f5ebe885e18>,
                                 vocabulary=None)),
                ('classifier',
                 LogisticRegression(C=1.0, class_weight=None, dual=False,
                                    fit_intercept=True, intercept_scaling=1,
             

In [ ]:
from sklearn import metrics
row = 33
# Predicting with a test dataset
predicted = pipe.predict(X_test)


print('material description was ----> ', X_test.iloc[row])
print('material predicted is ----->', predicted[row])

material description was ---->  BRAC ind reusable N building material
material predicted is -----> organic material


In [ ]:

# Model Accuracy
print("Accuracy:",metrics.accuracy_score(y_test, predicted ))
print("Precision:",metrics.precision_score(y_test, predicted, average='weighted'))
print("Recall:",metrics.recall_score(y_test, predicted, average='weighted'))


print(metrics.classification_report(y_test, predicted))

### Plot the description and material Outcomes

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
# plot
chosen_material = 'ground'


data = df.loc[df.material ==chosen_material]
sns.set_style('ticks')
fig, ax = plt.subplots()
fig.set_size_inches(1, 85)
sns.regplot(x='material', y='description', data=data, ax=ax)
